In [ ]:
5     1000005
12    1000012
19    1000019
21    1000021

In [3]:
import torch
import pandas as pd
import numpy as np
import torchio as tio
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
from zipfile import ZipFile
from zipfile import BadZipFile
import dask.dataframe as dd
import os
import multiprocessing as mp
import functools
from functools import partial
from os import path as pathOs
import comet_ml
from comet_ml import Experiment
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
#currently We want only imagfes with associated masks
df = df.loc[df['isAnyMissing'] ==False]
df = df.loc[df['isAnythingInAnnotated']>0 ]
# ignore all with deficient spacing
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]:    
    colName=keyWord+ "_spac_x"
    df = df.loc[df[colName]>0 ]
# get only complete representaions and only those with labels
df = df.loc[df['isAnyMissing'] ==False]
df = df.loc[df['isAnythingInAnnotated']>0 ]    
    
#just for testing    
#df=df.sample(n = 4)#TODO remove
df= df.head(4)
df

,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,patient_id,study_id,...,adc_stand,cor_stand,hbv_stand,sag_stand,adc_med_spac,hbv_med_spac,t2w_med_spac,label_med_spac,registered_adc_med_spac,registered_hbv_med_spac
0,0,0,0,0,0,5,5,5,10005,1000005,...,/home/sliceruser/data/orig/10005/10005_1000005...,/home/sliceruser/data/orig/10005/10005_1000005...,/home/sliceruser/data/orig/10005/10005_1000005...,/home/sliceruser/data/orig/10005/10005_1000005...,/home/sliceruser/data/orig/10005/10005_1000005...,/home/sliceruser/data/orig/10005/10005_1000005...,/home/sliceruser/data/orig/10005/10005_1000005...,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/orig/10005/10005_1000005...,/home/sliceruser/data/orig/10005/10005_1000005...
1,1,1,1,1,1,12,12,12,10012,1000012,...,/home/sliceruser/data/orig/10012/10012_1000012...,/home/sliceruser/data/orig/10012/10012_1000012...,/home/sliceruser/data/orig/10012/10012_1000012...,/home/sliceruser/data/orig/10012/10012_1000012...,/home/sliceruser/data/orig/10012/10012_1000012...,/home/sliceruser/data/orig/10012/10012_1000012...,/home/sliceruser/data/orig/10012/10012_1000012...,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/orig/10012/10012_1000012...,/home/sliceruser/data/orig/10012/10012_1000012...
2,2,2,2,2,2,19,19,19,10019,1000019,...,/home/sliceruser/data/orig/10019/10019_1000019...,/home/sliceruser/data/orig/10019/10019_1000019...,/home/sliceruser/data/orig/10019/10019_1000019...,/home/sliceruser/data/orig/10019/10019_1000019...,/home/sliceruser/data/orig/10019/10019_1000019...,/home/sliceruser/data/orig/10019/10019_1000019...,/home/sliceruser/data/orig/10019/10019_1000019...,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/orig/10019/10019_1000019...,/home/sliceruser/data/orig/10019/10019_1000019...
3,3,3,3,3,3,21,21,21,10021,1000021,...,/home/sliceruser/data/orig/10021/10021_1000021...,/home/sliceruser/data/orig/10021/10021_1000021...,/home/sliceruser/data/orig/10021/10021_1000021...,/home/sliceruser/data/orig/10021/10021_1000021...,/home/sliceruser/data/orig/10021/10021_1000021...,/home/sliceruser/data/orig/10021/10021_1000021...,/home/sliceruser/data/orig/10021/10021_1000021...,/home/sliceruser/labels/csPCa_lesion_delineati...,/home/sliceruser/data/orig/10021/10021_1000021...,/home/sliceruser/data/orig/10021/10021_1000021...


In [4]:
import JupyterNotebooksLib as slicernb
import slicer
slicer.mrmlScene.Clear(False)
# Load a Slicer sample data set
import SampleData
import SimpleITK as sitk
import sitkUtils

# Set viewer size to 50% of screen size
slicernb.AppWindow.setWindowSize(scale=0.5)
# Hide patient information from slice view
slicernb.showSliceViewAnnotations(False)

#slicernb.showVolumeRendering(volume, show=True)
app = slicernb.AppWindow()
app
app.setContents("full")
slicer.util.selectModule("Data")

def displayy(path,Name):    
    image=sitk.ReadImage(path)
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", Name)
    sitkUtils.PushVolumeToSlicer(image, outputVolumeNode)

from ipywidgets import HTML
HTML(f"""<a href="{slicernb.AppWindow.defaultDesktopUrl()}" target="_blank">
<b>Click here</b> to open application window in a new browser tab.</a>""")

HTML(value='<a href="/desktop/" target="_blank">\n<b>Click here</b> to open application window in a new browse…

In [5]:
#10005
reg_hbv = '/home/sliceruser/data/orig/10005/10005_1000005_hbv_stand_medianSpac_for_hbv_med_spac/result.0.mha'
t2w_resampl='/home/sliceruser/data/orig/10005/10005_1000005_t2w_stand_medianSpac.mha'
reg_adc=''
displayy(reg_hbv,"reg_hbv")
displayy(t2w_resampl,"t2w_resampl")


In [7]:
reg_hbv_image=sitk.ReadImage(reg_hbv)
t2w_resampl_image=sitk.ReadImage(t2w_resampl)
print(reg_hbv_image.GetSize())
print(t2w_resampl_image.GetSize())


(384, 384, 19)
(384, 384, 19)
